In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
  .master("local[*]") \
  .appName("test") \
  .config("spark.executor.instances", "8") \
  .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/13 09:10:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet("../../../data/pg/green/2020/01")

In [ ]:
SELECT 
  -- Reveneue grouping 
  date_trunc('hour', lpep_pickup_datetime) AS hour,
  PULocationID AS zone,

  SUM(total_amount) AS amount,
  COUNT(1) AS number_records
FROM green
WHERE lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1, 2
ORDER BY 1, 2

In [5]:
df_green.rdd.take(10)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), lpep_dropoff_datetime=datetime.datetime(2020, 1, 12, 18, 19, 52), store_and_fwd_flag='N', RatecodeID=1, PULocationID=41, DOLocationID=41, passenger_count=1, trip_distance=0.78, fare_amount=5.5, extra=0.0, mta_tax=0.5, tip_amount=1.58, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=7.88, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 20, 24, 10), lpep_dropoff_datetime=datetime.datetime(2020, 1, 31, 20, 31, 51), store_and_fwd_flag='N', RatecodeID=1, PULocationID=173, DOLocationID=70, passenger_count=1, trip_distance=0.98, fare_amount=7.0, extra=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=8.3, payment_type=2, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 8, 16, 53), lpep_dropoff_dat

In [12]:
df_green.take(10)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 4, 28, 15, 30, 20), lpep_dropoff_datetime=datetime.datetime(2020, 4, 28, 15, 37, 22), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=42, passenger_count=1, trip_distance=1.68, fare_amount=7.5, extra=0.0, mta_tax=0.5, tip_amount=1.66, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=9.96, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 4, 3, 17, 57, 32), lpep_dropoff_datetime=datetime.datetime(2020, 4, 3, 18, 6, 2), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=140, passenger_count=1, trip_distance=2.49, fare_amount=10.0, extra=1.0, mta_tax=0.5, tip_amount=1.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=15.55, payment_type=1, trip_type=1, congestion_surcharge=2.75),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 4, 6, 9, 18), lpep_dropoff_dat

In [3]:
rdd = df_green \
  .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
  .rdd
  

In [4]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 12, 18, 15, 4), PULocationID=41, total_amount=7.88),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 20, 24, 10), PULocationID=173, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 7, 8, 16, 53), PULocationID=74, total_amount=23.46),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 14, 47, 15), PULocationID=25, total_amount=7.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 31, 10, 8), PULocationID=259, total_amount=25.54)]

In [4]:
from datetime import datetime

start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
  return row.lpep_pickup_datetime >= start

In [5]:
rows = rdd.take(10)
row = rows[0]

In [6]:
row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)

datetime.datetime(2020, 1, 12, 18, 0)

In [7]:
def prepare_for_grouping(row):
  hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
  zone = row.PULocationID
  key = (hour, zone)
  
  amount = row.total_amount
  count = 1
  value = (amount, count)
  
  return (key, value)

In [8]:
rdd \
  .filter(filter_outliers) \
  .map(prepare_for_grouping) \
  .take(10)

[((datetime.datetime(2020, 1, 12, 18, 0), 41), (7.88, 1)),
 ((datetime.datetime(2020, 1, 31, 20, 0), 173), (8.3, 1)),
 ((datetime.datetime(2020, 1, 7, 8, 0), 74), (23.46, 1)),
 ((datetime.datetime(2020, 1, 15, 14, 0), 25), (7.3, 1)),
 ((datetime.datetime(2020, 1, 31, 10, 0), 259), (25.54, 1)),
 ((datetime.datetime(2020, 1, 18, 17, 0), 177), (13.3, 1)),
 ((datetime.datetime(2020, 1, 17, 20, 0), 65), (11.16, 1)),
 ((datetime.datetime(2020, 1, 13, 10, 0), 165), (20.56, 1)),
 ((datetime.datetime(2020, 1, 7, 15, 0), 170), (49.05, 1)),
 ((datetime.datetime(2020, 1, 10, 11, 0), 74), (9.56, 1))]

In [9]:
def calculate_revenue(left_value, right_value):
  left_amount, left_count = left_value
  right_amount, right_count = right_value
  
  output_amount = left_amount + right_amount
  output_count = left_count + right_count
  
  return (output_amount, output_count)


In [10]:
calculate_revenue((7.88, 1), (8.3, 1))

(16.18, 2)

In [13]:
def unwrap(row):
  return (row[0][0], row[0][1], row[1][0], row[1][1])

In [ ]:
rdd.filter(filter_outliers) \
  .map(prepare_for_grouping) \
  .reduceByKey(calculate_revenue) \
  .map(unwrap) \
  .toDF() \
  .take(10)


In [14]:
from collections import namedtuple

RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

def unwrap(row):
  return RevenueRow(
    hour = row[0][0], 
    zone = row[0][1], 
    revenue = row[1][0], 
    count = row[1][1])

In [15]:
from pyspark.sql import types

result_schema = types.StructType([
  types.StructField('hour', types.TimestampType(), True),
  types.StructField('zone', types.IntegerType(), True),
  types.StructField('revenue', types.DoubleType(), True),
  types.StructField('count', types.IntegerType(), True)
])

In [16]:
df_result = rdd.filter(filter_outliers) \
  .map(prepare_for_grouping) \
  .reduceByKey(calculate_revenue) \
  .map(unwrap) \
  .toDF(result_schema)

In [17]:
df_result.show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-12 18:00:00|  41|            916.74|   66|
|2020-01-13 10:00:00| 165| 55.43000000000001|    3|
|2020-01-23 10:00:00|  43|             315.4|   19|
|2020-01-02 09:00:00| 116|296.53999999999996|   17|
|2020-01-03 14:00:00| 242|163.61999999999998|   11|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-26 00:00:00| 258|             56.92|    1|
|2020-01-21 16:00:00|  33| 917.3600000000001|   38|
|2020-01-11 21:00:00| 244| 336.4700000000001|   10|
|2020-01-22 00:00:00|  22|             34.27|    1|
|2020-01-03 21:00:00|  74|            544.63|   41|
|2020-01-09 08:00:00|  42| 946.8600000000001|   69|
|2020-01-09 06:00:00| 124|             47.86|    2|
|2020-01-27 10:00:00|  25|            159.37|   11|
|2020-01-04 15:00:00|  25|457.37000000000006|   25|
|2020-01-24 01:00:00| 129|155.73000000000002|   12|
|2020-01-12 

In [ ]:
df_result.write.parquet("../../../data/tmp/green-revenue")

# TEST

In [3]:
from pyspark.sql import SparkSession
import datetime

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()


data = [((datetime.datetime(2020, 1, 12, 18, 0), 41), (7.88, 1)),
 ((datetime.datetime(2020, 1, 31, 20, 0), 173), (8.3, 1)),
 ((datetime.datetime(2020, 1, 7, 8, 0), 74), (23.46, 1)),
 ((datetime.datetime(2020, 1, 15, 14, 0), 25), (7.3, 1)),
 ((datetime.datetime(2020, 1, 31, 10, 0), 259), (25.54, 1)),
 ((datetime.datetime(2020, 1, 18, 17, 0), 177), (13.3, 1)),
 ((datetime.datetime(2020, 1, 17, 20, 0), 65), (11.16, 1)),
 ((datetime.datetime(2020, 1, 13, 10, 0), 165), (20.56, 1)),
 ((datetime.datetime(2020, 1, 7, 15, 0), 170), (49.05, 1)),
 ((datetime.datetime(2020, 1, 10, 11, 0), 74), (9.56, 1))]

rdd=spark.sparkContext.parallelize(data)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/13 08:09:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
rdd.take(100)

[((datetime.datetime(2020, 1, 12, 18, 0), 41), (7.88, 1)),
 ((datetime.datetime(2020, 1, 31, 20, 0), 173), (8.3, 1)),
 ((datetime.datetime(2020, 1, 7, 8, 0), 74), (23.46, 1)),
 ((datetime.datetime(2020, 1, 15, 14, 0), 25), (7.3, 1)),
 ((datetime.datetime(2020, 1, 31, 10, 0), 259), (25.54, 1)),
 ((datetime.datetime(2020, 1, 18, 17, 0), 177), (13.3, 1)),
 ((datetime.datetime(2020, 1, 17, 20, 0), 65), (11.16, 1)),
 ((datetime.datetime(2020, 1, 13, 10, 0), 165), (20.56, 1)),
 ((datetime.datetime(2020, 1, 7, 15, 0), 170), (49.05, 1)),
 ((datetime.datetime(2020, 1, 10, 11, 0), 74), (9.56, 1))]

In [5]:
def calculate_revenue(left_value, right_value):
  left_amount, left_count = left_value
  right_amount, right_count = right_value
  
  output_amount = left_amount + right_amount
  output_count = left_count + right_count
  
  return (output_amount, output_count)

In [6]:
# rdd2=rdd.reduceByKey(lambda a,b: a+b)
rdd2=rdd.reduceByKey(calculate_revenue)

for element in rdd2.collect():
    print(element)

((datetime.datetime(2020, 1, 7, 8, 0), 74), (23.46, 1))
((datetime.datetime(2020, 1, 7, 15, 0), 170), (49.05, 1))
((datetime.datetime(2020, 1, 17, 20, 0), 65), (11.16, 1))
((datetime.datetime(2020, 1, 10, 11, 0), 74), (9.56, 1))
((datetime.datetime(2020, 1, 12, 18, 0), 41), (7.88, 1))
((datetime.datetime(2020, 1, 31, 20, 0), 173), (8.3, 1))
((datetime.datetime(2020, 1, 31, 10, 0), 259), (25.54, 1))
((datetime.datetime(2020, 1, 13, 10, 0), 165), (20.56, 1))
((datetime.datetime(2020, 1, 15, 14, 0), 25), (7.3, 1))
((datetime.datetime(2020, 1, 18, 17, 0), 177), (13.3, 1))


In [13]:
rdd.top(5)

[((datetime.datetime(2020, 1, 31, 20, 0), 173), (8.3, 1)),
 ((datetime.datetime(2020, 1, 31, 10, 0), 259), (25.54, 1)),
 ((datetime.datetime(2020, 1, 18, 17, 0), 177), (13.3, 1)),
 ((datetime.datetime(2020, 1, 17, 20, 0), 65), (11.16, 1)),
 ((datetime.datetime(2020, 1, 15, 14, 0), 25), (7.3, 1))]

In [7]:
def unwrap(row):
  return (row[0][0], row[0][1], row[1][0], row[1][1])

In [8]:
rdd2=rdd.reduceByKey(calculate_revenue) \
  .map(unwrap) \
  .toDF()


In [9]:
rdd2.show()

+-------------------+---+-----+---+
|                 _1| _2|   _3| _4|
+-------------------+---+-----+---+
|2020-01-07 08:00:00| 74|23.46|  1|
|2020-01-07 15:00:00|170|49.05|  1|
|2020-01-17 20:00:00| 65|11.16|  1|
|2020-01-10 11:00:00| 74| 9.56|  1|
|2020-01-12 18:00:00| 41| 7.88|  1|
|2020-01-31 20:00:00|173|  8.3|  1|
|2020-01-31 10:00:00|259|25.54|  1|
|2020-01-13 10:00:00|165|20.56|  1|
|2020-01-15 14:00:00| 25|  7.3|  1|
|2020-01-18 17:00:00|177| 13.3|  1|
+-------------------+---+-----+---+

